# Finite Elements, Functions and Function Spaces


These tutorials will utilize Jupyter notebooks. Jupyter notebooks allow for markdown and code execution to be combined into one document, and are ideal for documentation and reproducibility. They also allow users to look at and experiment with FEniCS code. UFL documentation can be found [here](https://readthedocs.org/projects/fenics-ufl/downloads/pdf/stable/). See p.5 - 9 for discussions about finite elements.

In general, the goal is to solve for the displacement at the nodes during each time step that satisfies the balance of linear momentum. To do this we need to do the following:  

- Discretize the domain
- Describe the finite elements we want to use to solve our problem
- Describe the weak form we are trying to solve (and specify boundary conditions that aren't in the weak form. Remember, traction boundary conditions show up in the weak form, essential (displacement) boundary conditions are specified).
- Solve and update relevant quantities.

Now we can execute python code. This notebook should have been opened within the Jupyter Lab interface, started from the command line in an environment with access to dolfin. Let's start with importing numpy and dolfin:

In [1]:
import dolfin
import numpy as np
import fenics_plotly

# render the plots in this notebook
fenics_plotly.set_renderer("iframe")

A quick note about imports. In general, it is good practice to import packages, rather than `from dolfin import *`. Though a bit tedious, just importing packages will force you to precede function calls with the namespace they belong to. There are many overlapping functions between dolfin and ufl, and it is best to know which one you are using.

Let's start with a basic built-in unit cube mesh provide by FEniCS:

In [2]:
mesh = dolfin.UnitCubeMesh(1,1,1)
mesh2 = dolfin.UnitCubeMesh(10,8,6)

where the inputs define the refinement in the x, y, and z-directions respectively. These meshes can be saved and viewed in Paraview:

In [3]:
#File('mesh.pvd') << mesh
#File('mesh2.pvd') << mesh2
# But I'm not doing that yet, don't want to clutter the repository


In [61]:
fig1 = fenics_plotly.plot(mesh,color="red",opacity=0.5,show=False)
fig2 = fenics_plotly.plot(mesh2,color="blue",opacity=0.25,show=False)
fig1.show()

Mesh 2 is shown below. Note the refinement in x, y, and z. The rest of this tutorial will use the coarse unit cube mesh.

In [4]:
fig2.show()

Creating a Finite Element
--------------------------
Now to do anything interesting with our mesh, we need to define what type of finite elements we want to use. First, consider the definition of a finite element (Ciarlet 1975):  

*"A finite element is a triple (T, V, L), where:  
       - T is a closed, bounded subset of R^d with nonempty interior
         and piecewise smooth boundary.  
       - V = V(T) is a finite dimensional function space on T of dimension n  
       - L is the set of degrees of freedom (nodes) L = {l1,l2,...,ln} and
         is a basis for the dual space V' (space of bounded linear functionals
         on V)"*

More concretely, *T* gives us the discretization of our domain, *V* is the function space we use to approximate the solution on each of the subdomains (elements), and *L* is the evaluation of V on the nodes. Creating the unit cube mesh above, we have discretized our domain using tetrahedrons. To form a full finite element, we need to decide on a function space to approximate our solution. Let's start by considering a scalar quantity (say temperature) that we want to approximate as varying linearly within an element, and we want it to be continuous between elements. For this, we would use continuous Lagrange, linear polynomials. To create this type of basic finite element, we use:  

In [5]:
Q_elem = dolfin.FiniteElement("CG", mesh.ufl_cell(), 1, quad_scheme = "default")

Where "CG" stands for "Continuous Galerkin" (continuous between elements), "mesh.ufl_cell()" returns "tetrahedron", and "1" is the order of the polynomial (linear). With a tetrahedral geometry using linear polynomials, our degrees of freedom is the evaluation of the polynomials at the four vertices of the tetrahedron.  

 For our problem, we are trying to solve for displacement (a vector quantity) that we assume varies quadratically within an element. For this, we will use the quadratic Lagrange polynomials, and the shortcut command "VectorElement" which is technically a mixed element where all elements are equal:

In [6]:
V_elem = dolfin.VectorElement("CG", mesh.ufl_cell(), 2, quad_scheme = "default")

This is equivalent to declaring a basic quadratic CG element, and using it to declare a mixed element (we will see mixed elements later in our code):

In [7]:
Q_elem2 = dolfin.FiniteElement("CG", mesh.ufl_cell(), 2, quad_scheme = "default")
V_elem2 = Q_elem2 * Q_elem2 * Q_elem2

Creating a Finite Element Function Space
----------------------------------------
Defining the finite element gives a description of how the solution will be approximated *locally*, and then using the finite element and mesh we construct a global finite element function space:

In [8]:
Q_fcn_space = dolfin.FunctionSpace(mesh,Q_elem)
V_fcn_space = dolfin.FunctionSpace(mesh,V_elem)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


With a basis defined and the function space created, we can interpolate to get values anywhere within out geometry.

A quick note: The other type of element we use is called a "Quadrature Element". This element is used to obtain values ONLY AT THE ELEMENT QUADRATURE POINTS. In other words, you cannot interpolate using this element to get values at other coordinates. To do that, a projection must be done to a function space using one of the standard elements. MyoSim is solved using a quadrature element.

We can now define functions that belong to these finite element function spaces. Functions are useful to store the information we solve for along the way. For example, let's create a function that is meant to hold our displacement solution:  

In [9]:
u = dolfin.Function(V_fcn_space)

u is a FEniCS object, not just an array of numbers. If we want to view the array of function values, or store a copy of the function values, we can do <mark> it looks like .array() may be deprecated in the newer version. Just be consistent with .get_local() </mark>  

In [10]:
u.vector().get_local()
temp_u = u.vector().get_local()

We could also use the "get_local()" method, which gets the function values on the local process if things are being executed in parallel  

In [49]:
u.vector().get_local() #.reshape(27,3)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Let's just do a quick sanity check to verify the number of elements of our temp_u array. The function u belongs to a vector function space in our three dimensional mesh. Thus, for each node there should be three components (x, y, and z). We are using a basic unit cube mesh with refinement one, and quadratic tets. Thus there is a node at each vertex of a tetrahedron, and also at each midpoint. This leads to 27 nodes, each with 3 components, thus 81 elements in our temp_u array.

In [13]:
np.shape(u.vector().get_local())

(81,)

Now we can see function values, but for them to be useful we need a mapping between the indices of temp_u, and the coordinates of our mesh. Let's create that mapping:

In [43]:
gdim = mesh.geometry().dim() # get the dimension of our mesh. Will take on value of 3 for 3-dimensional mesh
V_dofmap = V_fcn_space.tabulate_dof_coordinates().reshape((-1,gdim)) # mapping comes from the function space
Q_dofmap = Q_fcn_space.tabulate_dof_coordinates().reshape((-1,gdim)) # Q is scalar function space, using only tet vertices
#V_dofmap
#Q_dofmap
np.shape(Q_dofmap)
#np.shape(V_dofmap)

(8, 3)

Notice the shape of Q_dofmap is (8,3). Since it's a scalar function space using linear tets, there should only be one value at each vertex (the four corners of the cube). Indeed, looking at Q_dofmap, it's a list of the coordinates representing the corners of the cube.

Assigning Function Values
------------------------
Let's create two functions on Q_fcn_space that are easier to deal with:

In [24]:
alpha = dolfin.Function(Q_fcn_space)
beta = dolfin.Function(Q_fcn_space)

There are a couple of ways to assign function values. You can change the elements of the function's .vector() attribute directly, or use the assign() function.  
You can assign individual elements of alpha (representing the function value at a specific vertex):  

In [40]:
alpha.vector()[4] = 4.0

Looking back at the Q_dofmap, we just assigned alpha to take on the value of 4.0 at the coordinate [0.,0.,0.]. Let's take a look at this function in Paraview to verify this.  

In [38]:
fig3 = fenics_plotly.plot(alpha,show=False)
fig3.show()
#alpha_file = File("alpha.pvd") # create paraview file
#alpha_file << alpha # save the function to the file

This verifies the assignment we have performed. Also note that because Q_fcn_space uses linear polynomials, alpha varies linearly from 4.0 at the origin, to 0.0 at the other vertices. We can also check the value of alpha at different points throughout our mesh. For this, we need to create a "point" object:  

In [35]:
p = dolfin.Point(0.0,1.0,1.0)
alpha(p)

4.0

evaluates alpha at the point p (where we assigned alpha), which comes out to 4. We can evaluate anywhere in the cube:

In [36]:
p1 = dolfin.Point(0.0,1.0,0.5) # along one edge
alpha(p1)

2.0000000000000004

We can also assign the full array of alpha values:

In [39]:
temp_alpha_values = np.linspace(0,np.shape(alpha.vector().get_local())[0],np.shape(alpha.vector().get_local())[0])
alpha.vector()[:] = temp_alpha_values
#alpha.vector()[:]
#alpha_file = File("alpha.pvd")
#alpha_file << alpha

array([0.        , 1.14285714, 2.28571429, 3.42857143, 4.57142857,
       5.71428571, 6.85714286, 8.        ])

Finally, we can assign one function to take on the value of another:

In [24]:
dolfin.assign(alpha,beta) #beta was initialized to zero, so we've reset alpha to be all zeros as well


It is encouraged to play with creating and assigning different function values to get used to using the degree of freedom mapping. It is also encouraged to create a new function space using discontinuous (DG) Lagrange elements (both of order 0 and 1), and to visualize your functions in ParaView to see if they match what you expect.  
Some final notes:  
- ParaView does not visualize above linear projections. In FEniCS, it is valid to create a scalar function space using continuous quadratic polynomials ("CG2"), but saving the function to a ParaView file will only save a CG1 projection.
- In our version of FEniCS, visualization at the integration points for quadrature elements is not implemented. This requires projecting from the quadrature function space to one of the others for visualization which may introduce errors. More of this will be discussed in a later tutorial.

It is more often the case that function values are assigned based on mathematical relationship, for example the stretch calculated from the Cauchy stretch tensor. This is one of the great things about FEniCS. One can (with things properly initialized) assign the relationship:

In [47]:
# f0 may have not been created above
# Let's create a vector valued function to hold the fiber direction
f0 = dolfin.Function(V_fcn_space) #later these will be defined at the quadrature points, for now at nodes
# Initialize f0 to be coincident with the x-direction

for i in np.arange(int(np.shape(f0.vector())[0]/3.0)):
    f0.vector()[i*3] = 1.0
# the vectors representing f0
print(f0.vector().get_local().reshape(27,3))

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [51]:
#alpha = dolfin.sqrt(dolfin.dot(f0, Cmat*f0))
#and as f0 (fiber direction) and the right Cauchy tensor are updated, alpha will be updated throughout the mesh as well.


<mark> Advise user to play around with different projections? </mark>

In [46]:
u.vector()[0]=1.0
u.vector()[:]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [69]:
Q_elemDG = dolfin.FiniteElement("DG", mesh.ufl_cell(), 1, quad_scheme = "default")
Q_DG_fcn_space = dolfin.FunctionSpace(mesh,Q_elemDG)
alphaDG = dolfin.Function(Q_DG_fcn_space)


In [70]:
alphaDG.vector()[4] = 4.0
fig4 = fenics_plotly.plot(alphaDG,show=False)
fig4.show()


In [71]:
p2 = dolfin.Point(0.5,0.0,0.0) # along one edge
alphaDG(p2)

0.0

In [72]:
p3 = dolfin.Point(1.0,0.0,0.5) # along one edge
alphaDG(p3)

2.0

In [73]:
p4 = dolfin.Point(1.0,0.5,0.0) # along one edge
alphaDG(p4)

0.0